> 参考：
> 
> https://blog.csdn.net/qq_27825451/article/details/84581272

# 一、装饰器Decorator基础
装饰器（decorators）是 Python 中的一种高级功能，它允许你动态地修改函数或类的行为。

装饰器是一种函数，它接受一个函数作为参数，并返回一个新的函数或修改原来的函数。

装饰器的语法使用 @decorator_name 来应用在函数或方法上。

Python 还提供了一些内置的装饰器，比如 `@staticmethod` 和` @classmethod`，用于定义静态方法和类方法。

装饰器的应用场景：

- 日志记录: 装饰器可用于记录函数的调用信息、参数和返回值。
- 性能分析: 可以使用装饰器来测量函数的执行时间。
- 权限控制: 装饰器可用于限制对某些函数的访问权限。
- 缓存: 装饰器可用于实现函数结果的缓存，以提高性能。

## 先从一种情况开始看起
### 1、装饰器decorator的由来  
装饰器的定义很是抽象，我们来看一个小例子。  
先定义一个简单的函数：  

In [2]:
def myfunc():
    print('我是函数myfunc')
 
myfunc()  #调用函数

我是函数myfunc


In [5]:
# 然后呢，我想看看这个函数执行这个函数用了多长时间，好吧，那么我们可以这样做：
import time

def myfunc():
    start = time.perf_counter()
    print('我是函数myfunc')
    end = time.perf_counter()
    print('函数myfunc执行了%s秒' % (end - start))

myfunc()

我是函数myfunc
函数myfunc执行了0.0002133999987563584秒


我们现在已经达到了我们的目的。但是如果是我们还想继续给另外的一些函数也实现同样的功能。那我们是不是给每个函数都添加这么几句话呢？当然可以，但是不高效，而且很麻烦。如果有某一种方式可以一次性解决所有的问题，那自然最好不过了，于是“装饰器”就应运而生。

在上面的例子中，函数本身的功能只是打印一句话而已，但是经过改造后的函数不仅要能够打印这一句话，还要能够显示函数执行所花费的时间，这相当于我要给这个函数添加额外的功能，注意这个关键字，其实“装饰器”就是专门给函数添加额外的功能的。

### 2、添加额外功能的简单实现——非“装饰器”实现
还记得吗，函数在Python中是一等公民，那么我们可以考虑重新定义一个函数timeit，将myfunc的引用传递给他，然后在timeit中调用myfunc并进行计时，这样，我们就达到了不改动myfunc定义但是又添加了额外功能的目的，代码如下：


In [6]:
import time

def myfunc():
    print("我是函数myfunc")

def timeit(function):
    start = time.perf_counter()
    function()
    end = time.perf_counter()
    print("函数执行时间：", end - start)
    return function

timeit(myfunc)

我是函数myfunc
函数执行时间： 0.00020460000087041408


<function __main__.myfunc()>

## 什么是装饰器——decorator
一般而言，如果我需要给函数添加额外的某一些功能，我需要修改函数的源代码，但是如前面所说，这样麻烦，而且不高效，装饰器就是专门的解决方案！

### 1、什么是装饰器？——两个层面

在Python里面有两层定义：

第一：从设计模式的层面上

装饰器是一个很著名的设计模式，经常被用于有**切面**需求的场景，较为经典的应用有插入日志、增加计时逻辑来检测性能、加入事务处理等。装饰器是解决这类问题的绝佳设计，有了装饰器，我们就可以抽离出大量函数中与函数功能本身无关的雷同代码并继续重用。概括的讲，**装饰器的作用就是为已经存在的对象添加额外的功能**。

第二：从Python的语法层面上（其实第二种本质上也是第一种，只不过在语法上进行了规范化）

简言之，python装饰器就是用于拓展原来函数功能的一种函数，这个函数的特殊之处在于它的返回值也是一个函数，使用python装饰器的好处就是在不用更改原函数的代码前提下给函数增加新的功能。 如此一来，我们要想拓展原来函数代码，就不需要再在函数里面修改源代码了。

### 2、装饰器的作用——两方面
（1）抽离雷同代码，加以重用

（2）为函数添加额外的功能


### 3、装饰器的使用场景

（1）缓存装饰器

（2）权限验证装饰器

（3）计时装饰器

（4）日志装饰器

（5）路由装饰器

（6）异常处理装饰器

（7）错误重试装饰器

## 装饰器的实现
### 1、装饰器的逐步实现

针对上面改进版的代码所存在的哪些问题，我们想出了解决办法：

既然修改N处的调用代码很麻烦，我们就来想想办法不修改调用代码；如果不修改调用代码，也就意味着调用myfunc()需要产生调用timeit(myfunc)的效果。

因为python中一切皆对象，故而我们可以想到将timeit赋值给myfunc：

In [ ]:
import time
 
def myfunc():
 
    print("我是函数myfunc")
 
def timeit(function):
    start = time.perf_counter()
    function()
    end =time.perf_counter()
    print(f'函数执行所花费的时间为：{end-start}')
 
myfunc=timeit  #将timeit赋值给原来的myfunc
myfunc()

但是上面的调用并不会成功，会显示出如下错误：

timeit() missing 1 required positional argument: 'function'

这是因为将timeit赋值给myfunc之后，此时myfunc和timeit表示的同一个东西，但是timeit似乎带有一个参数function，而在调用myfunc()的时候并没有传入任何参数，所以并不会成功。

但是上面的调用虽然没有成功，却给我们指出了一条重要的线索，因为上面的代码已经解决“修改调用代码”的问题，只不过是参数没有统一而已，那就想办法把参数统一吧！那就再添加一个函数呗！什么意思？

因为参数不统一，如果timeit()不并不是直接添加额外的功能，而是返回一个与myfunc参数列表一致的函数。而原来timeit需要添加额外功能的代码再在timeit里面定义一个函数，由它去完成不就可以了吗，将timeit(myfunc)的返回值赋值给myfunc，然后，调用myfunc()的代码完全不用修改。——即我们依然是调用myfunc（调用代码没变），但是同样却达到了添加额外功能的效果！


In [9]:
import time
#原来的函数myfunc
def myfunc():
    print("我是函数myfunc")
 
#定义一个计时器
def timeit(function):
    '''
       timeit函数负责返回一个wrapper，wrapper的参数要与原来的myfunc保持相同
       这样一来，执行 myfunc=timeit(myfunc)  myfunc完全等价于wrapper
       wrapper函数负责添加额外功能
    '''
    def wrapper():
        start = time.perf_counter()
        function()
        end =time.perf_counter()
        print(f'函数执行所花费的时间为：{end-start}')
    return wrapper
 
myfunc=timeit(myfunc)  #注意，这里与前面的 “myfunc=timeit”是有所区别的哦
myfunc()  #还和原来调用myfunc()一样,但是达到了添加额外功能的效果

我是函数myfunc
函数执行所花费的时间为：0.00023720000172033906


上面的运行结果就出来了，如下：

我是函数myfunc  
函数执行所花费的时间为：0.0005973331798019136  
总结：在上面的函数定义和调用中，看起来我的调用myfunc()和原来并没有任何不同，但是却已经添加了额外的效果。它解决前面存在的两个问题：  

（1）不用修改函数源代码，也不用修改调用函数的代码，完全跟调用最原始的myfunc()代码一样，但是却添加了额外功能；

（2）解决了timeit和myfunc的参数不统一问题，那就是再添加一层wrapper；

——这就是装饰器。

上面的装饰器就是最原始的版本，但是python中引入了专门的“语法糖”来实现装饰器，这样看起来更加专业，更加美观。就是使用字符“@”去实现。代码如下：

In [10]:
import time
 
#定义一个计时器
def timeit(function):
    '''
       timeit函数负责返回一个wrapper，wrapper的参数要与原来的myfunc保持相同
       这样一来，执行 myfunc=timeit(myfunc)  myfunc完全等价于wrapper
       wrapper函数负责添加额外功能
    '''
    def wrapper():
        start = time.perf_counter()
        function()
        end =time.perf_counter()
        print(f'函数执行所花费的时间为：{end-start}')
    return wrapper
 
#myfunc=timeit(myfunc)  #注意，这里与前面的 “myfunc=timeit”是有所区别的哦
 
#原来的函数myfunc
@timeit
def myfunc():
    print("我是函数myfunc")
 
myfunc()  #还和原来调用myfunc()一样,但是达到了添加额外功能的效果

我是函数myfunc
函数执行所花费的时间为：0.00020109999968553893


上面代码的运行结果依然是：

我是函数myfunc
函数执行所花费的时间为：0.0004893814003196401

在上面的例子中，在定义myfunc函数的上面加了一个@timeit，这与前面的写法myfunc = timeit(myfunc)完全等价，

**@有两个重要的作用，第一：较少了代码书写量；第二：那就是让我们的代码看上去更有装饰器的感觉，看起来更高端了。**

总结：

在这个例子中，函数进入和退出时需要计时，这被称为一个**横切面(Aspect)**，这种编程方式被称为**面向切面的编程(Aspect-Oriented Programming)**。与传统编程习惯的从上往下执行方式相比较而言，像是在函数执行的流程中横向地插入了一段逻辑。在特定的业务领域里，能减少大量重复代码。面向切面编程还有相当多的术语，这里就不多做介绍，感兴趣的话可以去找找相关的资料（如果有需要，我后面也会抽时间专门写一系列关于面向切面编程的文章，看我有没有时间啦！）

### 2、装饰器的一般结构
为了能够明确装饰器的实现原理，这里给出一个关于装饰器的“一般模板”，方便大家理解！但是，装饰器作为一种设计模式，本身是没有固定的设计模板的，语法也是相对较为灵活，没有说一定要怎么写才正确。

```python
def decorator(function):
    '''
    第一层函数为装饰器名称
    function：参数，即需要装饰的函数
    return：返回值wrapper，为了保持与原函数参数一致
    '''
    def wrapper(*arg,**args):
        '''
           内层函数，这个函数实现“添加额外功能”的任务
           *arg,**args：参数保持与需要装饰的函数参数一致，这里用*arg和**args代替
        '''
        #这里就是额外功能代码
        function()   #执行原函数
        #这里就是额外功能代码
    return wrapper
```


# 二、函数装饰器、类装饰器详解
## 函数装饰器
前面提到过，装饰器分为函数装饰器、类装饰器，本节详细解释函数装饰器，又分为两种情况，因为函数装饰器可以装饰函数，也可以装饰类。函数装饰器是最常见的，故而最先讨论。
注意：因为没有参数，且无函数返回值的函数是最为简单的，就如“上篇”所讨论的那样，这里就不再叙述了，本文主要讲的都是函数带有参数，而且具有函数返回值的函数。

### 1、函数装饰器 装饰一个函数
假定有一个函数实现两个数的相加，a+b，但是为了对这两个数相加的结果进行加密，我们需要给函数添加额外的代码，但是我们通过装饰器去实现，要达到的效果是，不是直接返回a+b的结果，而是进行进一步处理。代码如下：



In [11]:
def MethodDecoration(function):  #外层decorator
    c=150
    d=200
    def wrapper(a,b):            #内层wrapper。和add_function参数要一样
        result=function(a,b)
        result=result*c/d        #加密，相当于添加额外功能
        return result            #此处一定要返回值
    return wrapper
 
@MethodDecoration
def add_function(a,b):
    return a+b
 
 
result=add_function(100,300)    #函数调用
print(result)

300.0


运行结果为：300  （即(100+300)*150/200）

因为函数装饰器去装饰函数最为常见，所以这里就不多再解释了，按照前面上篇所讲的模板来即可，但是因为被装饰的函数有参数，而且具有返回值，有两个点需要注意的，

第一：wrapper需要保证与add_function参数一致。因为返回的wrapper就是add_function，所以要统一，我们可以使用*arg,和**args去匹配任何参数；

第二：wrapper一定要返回值。因为add_function函数是需要返回值的。


In [26]:
# 写法1
def MethodDecoration(function):
    c = 150
    d = 200
    def wrapper(*args):
        result=function(*args)
        result=result*c/d   
        return result
    return wrapper

@MethodDecoration
def add_function(a, b):
    return a + b

result = add_function(100, 300)    #函数调用
print(result)

300.0


In [24]:
# 写法2
def MethodDecoration(function):
    c = 150
    d = 200
    def wrapper(*args):
        result=function(*args)
        result=result*c/d   
        return result
    return wrapper
    
@MethodDecoration
def add_function(*args):
    total = 0
    for num in args:
        total += num
        
    return total


result = add_function(100, 300)    #函数调用
print(result)

300.0


### 2、函数装饰器 装饰一个类

在Python中，从某种意义上来说，函数和类是一样的，因为它们都是对象（python一切皆对象），故而decorator的参数理所当然也可以传入一个类了。其中最经典的应用，就是使用装饰器构造“单例模式”（不明白单例模式的小伙伴可以参见下面这篇博文哦）

[一文详解“单例模式”及其python语言的实现](https://blog.csdn.net/qq_27825451/article/details/84135466)

In [32]:
def Singleton(cls): #这是第一层函数，相当于模板中的Decorator.目的是要实现一个“装饰器”，而且是对类型的装饰器
    '''
    cls:表示一个类名，即所要设计的单例类名称，
        因为python一切皆对象，故而类名同样可以作为参数传递
    '''
    instance = {}

    def singleton(*args, **kwargs): #这是第二层函数，相当于模板中的Decorator_func.目的是要实现一个“装饰器函数”，
        '''
        *args:表示函数参数列表，
        **kwargs:表示函数关键字参数列表
        '''
        if cls not in instance: #如果单例类还没有被实例化过，则进行实例化
            instance[cls] = cls(*args, **kwargs) # #如果没有cls这个类，则创建，并且将这个cls所创建的实例，保存在一个字典中
        return instance[cls] # 返回创建的对象
    
    return singleton
    
@Singleton
class Student(object):
    def __init__(self, name,age):
        self.name=name
        self.age=age

s1 = Student('张三',23)
s2 = Student('李四',24)
print((s1==s2))
print(s1 is s2)
print(id(s1),id(s2),sep='   ')

print(s1.name)
print(s2.name)

True
True
2077898510864   2077898510864
张三
张三


上面运行的结果是：

True
True
2150787003840   2150787003840
张三
张三

懂得单例模式的小伙伴可能一看就明白了，上面的实现和我前面讲过的“装饰器模板”，基本上一样，第一层、第二层、返回值、参数匹配等。但是有的小伙伴可能会问，这里我没有看到“添加额外功能”啊，装饰器不是添加额外功能的么？实际上“添加额外功能”是一种抽象的表述，不是说一定要添加什么东西，**对被装饰的对象（函数、类）进行某种约束、处理、添加、删减等额外操作统称为添加额外功能**。

这里约束了这个类型Student的创建，主要这个类还没有创建实例，就创建一个，只要创建了，就不在创建新的实例了，只需要把之前创建的返回即可，这是单例模式的思想。如果还是不明白，下面再举一个“添加额外功能”的例子。

比如我有一个学生类，在创建学生实例的时候有两个实例属性，name，age，现在要通过装饰器对类加以装饰，使得在创建学生类的实例的时候，还会添加height和weight两个属性，代码如下：
